# Recomendacao de Sistemas Baseado em Grafos
### Análise de Algoritmos e Estrutura de Dados -  Profa. Dra. Lilian Berton
### Nome: Willian Dihanster Gomes de Oliveira

### Resumo:
Recomendação de sistemas possuem diversas aplicações no mundo real e tem sido um tema de grande interesse para grandes empresas. Diversas abordagens já foram propostas, incluindo métodos baseados em grafos bipartidos, que são capazes de modelar a relação usuários e itens de forma prática e intuitiva. Assim, neste trabalho, foi realizada a implementação de um sistema de recomendação baseado em grafos para recomendação de filmes aos usuários de um site. Os resultados obtidos foram comparados com dois métodos clássicos (k-popularidade e aleatório) em termos de coverage e personalization e indicaram bons resultados.

Referências (Código-fonte): <br>
https://github.com/kurasaiteja/Github-Recommender-System
https://github.com/statisticianinstilettos/recmetrics/blob/e8d9b39131999c484c4c98265f91e233be6ca4cb/recmetrics/metrics.py#L160

## 1 - Leitura dos Datasets

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import random
import scipy.sparse as sp
import time

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Carrega o dataset de movies
movies = pd.read_csv('./data/movies.csv', sep=',')
movies['movieId'] = movies['movieId'] * 1000
movie_ids = list(movies.movieId.unique()) 
print(movies.head())

   movieId                               title                                       genres
0     1000                    Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1     2000                      Jumanji (1995)                   Adventure|Children|Fantasy
2     3000             Grumpier Old Men (1995)                               Comedy|Romance
3     4000            Waiting to Exhale (1995)                         Comedy|Drama|Romance
4     5000  Father of the Bride Part II (1995)                                       Comedy


In [ ]:
# Carrega o dataset de ratings
ratings = pd.read_csv('./data/ratings.csv', sep=',')
ratings['movieId'] =  ratings['movieId'] * 1000
user_ids = list(ratings.userId.unique())
print(ratings.head())

   userId  movieId  rating   timestamp
0       1    31000     2.5  1260759144
1       1  1029000     3.0  1260759179
2       1  1061000     3.0  1260759182
3       1  1129000     2.0  1260759185
4       1  1172000     4.0  1260759205


In [ ]:
train = pd.DataFrame()
test = pd.DataFrame()

for i in np.unique(ratings['userId']):
    train = train.reset_index(drop=True)
#    example_train, example_test = train_test_split(ratings[ratings['userId']==i].sort_values(by='timestamp'), train_size=0.9, shuffle=False)
    example_train, example_test = train_test_split(ratings[ratings['userId']==i], train_size=0.7, shuffle=False)
    example_train = example_train.reset_index(drop=True)
    example_test = example_test.reset_index(drop=True)
    
    if len(train) == 0:
        train = pd.DataFrame(example_train, columns=ratings.columns)
        test = pd.DataFrame(example_test, columns=ratings.columns)
    else:
        train = pd.concat([train, example_train], axis=0, ignore_index=True)
        test = pd.concat([test, example_test], axis=0, ignore_index=True)
        
ratings = train.copy()

## 2 - Criacao do Grafo Bipartido

In [ ]:
# Criar um grafo
print('Criando Grafo Bipartido...')
G = nx.Graph()

# Criar os nohs do grafo, onde nohs = user_ids e movies_ids
G.add_nodes_from(user_ids, bipartite='users')
G.add_nodes_from(movie_ids, bipartite='movies')

# Cria as arestas do grafo, onde user_ids se relacionam com filmes
# Dado que o grafo eh bipartido, user_ids soh se relacionam com filmes e vice-versa
for name, group in ratings.groupby(['userId', 'movieId']):
    userId, movieId = name
    G.add_edges_from([(userId, movieId)])

# Printa o numero de vertices e arestas
print('Grafo Bipartido de Usuarios x Filmes criado!')
print(f'Grafo possui {len(G.nodes())} nohs e {len(G.edges())} arestas')

Criando Grafo Bipartido...
Grafo Bipartido de Usuarios x Filmes criado!
Grafo possui 9796 nohs e 69717 arestas


## 3 - Recomendador de Sistemas 

### 3.1 - Nohs de uma Particao

In [ ]:
# Retorna todos os noh de uma particao
def get_nodes_from_partition(G, partition):
    nodes = []
    
    # Para cada noh no Grafo
    for node in G.nodes():
        # Se a particao do noh atual == partition, adicionamos o noh
        if G.nodes[node]['bipartite'] == partition:
            nodes.append(node)
            
    return nodes

# Printa o numero de noh nas particoes
print(len(get_nodes_from_partition(G, 'users')))
print(len(get_nodes_from_partition(G, 'movies')))

671
9125


### 3.2 - Filmes em Comum entre dois Nohs

In [ ]:
# Retorna filmes em comum entre dois nohs
def shared_partition_nodes(G, node1, node2):
    # Assegura que os nos estao na mesma particao
    assert G.nodes[node1]['bipartite'] == G.nodes[node2]['bipartite']

    # Pega os vizinhos dos nohs
    nbrs1 = G.neighbors(node1)
    nbrs2 = G.neighbors(node2)

    # Calcula interseccao de de filmes entre os nohs
    overlap = set(nbrs1).intersection(nbrs2)
    return overlap

# Printa os filmes em comum entre dois nohs
node1 = 1
node2 = 4
print(f'Filmes em comum com o noh {node1} e noh {node2}:')
for m in shared_partition_nodes(G, node1, node2):
    print(np.array(movies[movies['movieId'] == m][['movieId', 'title']]).ravel())

Filmes em comum com o noh 1 e noh 4:
[1371000 'Star Trek: The Motion Picture (1979)']
[1953000 'French Connection, The (1971)']
[2105000 'Tron (1982)']


### 3.3 - Calculando Similaridade entre Usuarios

In [ ]:
# Retorna um coeficiente de similaridade entre dois nohs
def user_similarity(G, user1, user2, movies_nodes):
    # Assegura que os nohs sao da particao "users"
    assert G.nodes[user1]['bipartite'] == 'users'
    assert G.nodes[user2]['bipartite'] == 'users'

    # Pega o conjunto de filmes compartilhados entre dois usuarios
    shared_nodes = shared_partition_nodes(G, user1, user2)
    
    # Retorna o coeficiente de similaridade entre os nohs
    # Dado pelo numero de filmes compartilhados pelo numero de filmes totais
    return len(shared_nodes) / len(movies_nodes)

# Computa o coeficiente de similaridade entre dois usuarios
movies_nodes = get_nodes_from_partition(G, 'movies')
node1, node2 = 1, 73
similarity_score = user_similarity(G, node1, node2, movies_nodes)
print(f'Score de Similaridade entre o noh {node1} e noh {node2} eh {similarity_score}')

Score de Similaridade entre o noh 1 e noh 73 eh 0.0012054794520547946


In [ ]:
# Retorna os usuarios mais similares a um noh
def most_similar_users(G, user, user_nodes, proj_nodes):
    # Assegura que o noh eh um 'user'
    assert G.nodes[user]['bipartite'] == 'users'

    # Pega todos os outros usuarios
    user_nodes = set(user_nodes)
    user_nodes.remove(user)

    # Cria um dicionario de similaridades
    similarities = defaultdict(list)
    for node in user_nodes:
        similarity = user_similarity(G, user, node, proj_nodes)
        similarities[similarity].append(node)

    # Calcula a maior similaridade
    max_similarity = max(similarities.keys())
    
    # Retorna lista de usuarios com maior similaridade ao noh
    return similarities[max_similarity]

user_nodes = get_nodes_from_partition(G, 'users')
movies_nodes = get_nodes_from_partition(G, 'movies')
node1 = 1
print(f'Usuarios mais similares ao noh {node1} sao: {most_similar_users(G, node1, user_nodes, movies_nodes)}')

Usuarios mais similares ao noh 1 sao: [73, 468]


### 3.4 - Recomendador de Filmes

In [ ]:
# Faz a recomendacao de filmes para um usuario, dado um usuario similar
def recommend_movies(G, from_user, to_user):
    # Pega os filmes que o usuario semelhante assistiu e os que o usuario assistiu
    from_movies = set(G.neighbors(from_user))
    to_movies = set(G.neighbors(to_user))

    # Retorna os filmes que o usuario ainda nao assistiu, e o usuario semelhante sim
    return from_movies.difference(to_movies)

# Printa os filmes as serem recomendados
node1, node2 = 1, 73
print(f'Filmes recomendados ao noh {node1} sao: {recommend_movies(G, node1, node2)}')
for m in recommend_movies(G, node1, node2):
    print(np.array(movies[movies['movieId'] == m][['movieId', 'title']]).ravel())

Filmes recomendados ao noh 1 sao: {1172000, 1287000, 1371000}
[1172000 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)']
[1287000 'Ben-Hur (1959)']
[1371000 'Star Trek: The Motion Picture (1979)']


### 3.5 - Metricas de Desempenho

In [ ]:
def prediction_coverage(predicted, catalog):
    predicted_flattened = [p for sublist in predicted for p in sublist]
    unique_predictions = len(set(predicted_flattened))
    prediction_coverage = round(unique_predictions/(len(catalog)* 1.0)*100, 2)
    
    return prediction_coverage

def personalization(predicted):
    predicted = np.array(predicted)
    
    df = pd.DataFrame(data=predicted).reset_index().melt(
        id_vars='index', value_name='item',
    )
    
    a = len(dic)
    b = len(np.unique(train.movieId))
    df = pd.DataFrame(np.zeros((a, b)), columns=list(np.unique(train.movieId)))
    for d in dic:
        for c in predicted[d-1]:
            df.loc[df.index==d, c] = 1  
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    rec_matrix_sparse = sp.csr_matrix(df.values)
    
    similarity = cosine_similarity(X=rec_matrix_sparse, dense_output=False)
    upper_right = np.triu_indices(similarity.shape[0], k=1)
    personalization = 1 - np.mean(similarity[upper_right])
    
    return personalization

### 3.5 - Uso dos Recomendadores

In [ ]:
recommenders = ['Grafo', 'Aleatorio', 'Populares']

for r in recommenders:
    preds = []
    trues = []
    coverage = []
    dic = {}
    for id in np.unique(ratings.userId):
        dic[id] = []
    k = 10
    
    print(f'Iniciando recomendacao do tipo {r} com {k} recomendacoes')
    inicio = time.time()
    for d in np.unique(ratings.userId):
        
        if r == 'Grafo':
            similars = most_similar_users(G, d, user_nodes, movies_nodes)
            for s in similars:
                movies_recommend = recommend_movies(G, d, s)
                dic[d] += movies_recommend
                
            p = list(np.unique(dic[d]))
        elif r == 'Aleatorio':
            p = list(np.unique(movies['movieId']))
        elif r == 'Populares':
            p = list(ratings['movieId'].value_counts().sort_values(ascending=False)[0:k])
            
        random.shuffle(p)
        preds.append(p[0:k]) 
        trues.append(list(test[test['userId']==d]['movieId']))
    
    fim = time.time()
    print(f'Tempo gasto com recomendacao do tipo {r} = {fim-inicio}s')        

    
    cov = prediction_coverage(preds, np.unique(movies['movieId']))
    ind_pers = personalization(preds)
    print(f"Resultados obtidos: \n Coverage = {cov} e Indice de Personalizacao = {ind_pers}")

Iniciando recomendacao do tipo Grafo com 10 recomendacoes
Tempo gasto com recomendacao do tipo Grafo = 9.815768003463745s
Resultados obtidos: 
 Coverage = 22.96 e Indice de Personalizacao = 0.9922962532824188
Iniciando recomendacao do tipo Aleatorio com 10 recomendacoes
Tempo gasto com recomendacao do tipo Aleatorio = 6.693778991699219s
Resultados obtidos: 
 Coverage = 52.44 e Indice de Personalizacao = 0.9989087350134573
Iniciando recomendacao do tipo Populares com 10 recomendacoes
Tempo gasto com recomendacao do tipo Populares = 3.2840988636016846s
Resultados obtidos: 
 Coverage = 0.11 e Indice de Personalizacao = 0.0029806259314457684


## 4 - Conclusoes
Neste trabalho objetivamos construir uma recomendador de sistemas, para recomendar filmes, em um modelo baseado em grafos bipartidos. Os resultados obtidos, dado as métricas de objetivo, foram bons, apresentando bons valores de \textit{coverage} e \textit{personalization}.

Entretanto, com essas métricas, o modelo não foi melhor que o modelo Random. isso é esperado, dado a definição das métricas e o modelo Random, que é beneficiado pelas propriedades aleatórias. É esperado que o modelo Random tenha baixos resultados de acurácia, e com isso, o modelo Graph-based pode ser mais indicados.

Dessa forma, mais experimentos, incluindo novas métricas poderiam ser realizados. Além disso, também pode ser testado novas métricas para cálculo de similaridade entre usuários, que é um dos principais "parâmetros" do algoritmo baseado em grafos bipartidos.

